In [21]:
# importing relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

In [3]:
import pandas as pd
df = pd.read_csv('zillow_data.csv')
display(df.head())
df.iloc[:,0:7].nunique()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


RegionID      14723
RegionName    14723
City           7554
State            51
Metro           701
CountyName     1212
SizeRank      14723
dtype: int64

In [22]:
ny = df[df['City'] == 'New York']
ny.rename({'RegionName': 'Zip'}, axis = 1, inplace = True)
ny.reset_index(drop=True, inplace = True)

In [23]:
ny

,RegionID,Zip,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
1,62037,11226,New York,NY,New York,Kings,11,162000.0,162300.0,162600.0,...,860200,851000,853900,870000,885100,887800,890500,901700,930700,963200
2,62087,11375,New York,NY,New York,Queens,13,252400.0,251800.0,251400.0,...,1022600,1033700,1048600,1066400,1081200,1088800,1092700,1089500,1084000,1084600
3,62045,11235,New York,NY,New York,Kings,14,190500.0,191000.0,191500.0,...,767300,777300,788800,793900,796000,799700,806600,810600,813400,816200
4,61625,10011,New York,NY,New York,New York,21,NaN,NaN,NaN,...,12137600,12112600,12036600,12050100,12016300,11946500,11978100,11849300,11563000,11478300
5,61703,10128,New York,NY,New York,New York,22,3676700.0,3704200.0,3729600.0,...,7410100,7422400,7417600,7427300,7371400,7342700,7353300,7350300,7363000,7386600
6,61802,10462,New York,NY,New York,Bronx,36,154300.0,154300.0,154400.0,...,432800,436100,439900,443500,444100,443900,446000,446400,447200,450200
7,61796,10456,New York,NY,New York,Bronx,37,NaN,NaN,NaN,...,357900,357100,356500,357200,362000,368500,374100,379800,388100,394400
8,61803,10463,New York,NY,New York,Bronx,47,180100.0,181000.0,181900.0,...,680800,684500,692000,700900,710200,726900,744500,757000,769400,778700
9,62040,11230,New York,NY,New York,Kings,48,230100.0,230800.0,231500.0,...,1079200,1082700,1082800,1084900,1091200,1102400,1120200,1141600,1170000,1195400


In [24]:
Bronx = [10453, 10457, 10460, 10458, 10467, 10468, 10451, 10452,
         10456, 10454, 10455, 10459, 10474, 10463, 10471, 10466,
         10469, 10470, 10475, 10461, 10462,10464, 10465, 10472, 10473]
Brooklyn = [11212, 11213, 11216, 11233, 11238, 11209, 11214, 11228,
            11204, 11218, 11219, 11230, 11234, 11236, 11239, 11223,
            11224, 11229, 11235, 11201, 11205, 11215, 11217, 11231,
            11203, 11210, 11225, 11226, 11207, 11208, 11211, 11222,
            11220, 11232, 11206, 11221, 11237]
Manhattan = [10026, 10027, 10030, 10037, 10039, 10001, 10011, 10018,
             10019, 10020, 10036, 10029, 10035, 10010, 10016, 10017,
             10022, 10012, 10013, 10014, 10004, 10005, 10006, 10007,
             10038, 10280, 10002, 10003, 10009, 10021, 10028, 10044,
             10065, 10075, 10128, 10023, 10024, 10025, 10031, 10032,
             10033, 10034, 10040]
Queens = [11361, 11362, 11363, 11364, 11354, 11355, 11356, 11357,
          11358, 11359, 11360, 11365, 11366, 11367, 11412, 11423,
          11432, 11433, 11434, 11435, 11436, 11101, 11102, 11103,
          11104, 11105, 11106, 11374, 11375, 11379, 11385, 11691,
          11692, 11693, 11694, 11695, 11697, 11004, 11005, 11411,
          11413, 11422, 11426, 11427, 11428, 11429, 11414, 11415,
          11416, 11417, 11418, 11419, 11420, 11421, 11368, 11369,
          11370, 11372, 11373, 11377, 11378]
StatenIsland = [10302, 10303, 10310, 10306, 10307, 10308, 10309,
                10312, 10301, 10304, 10305, 10314]
bx = ny.copy()
bx.drop(ny.index, inplace=True)
bk = ny.copy()
bk.drop(ny.index, inplace=True)
man = ny.copy()
man.drop(ny.index, inplace=True)
que = ny.copy()
que.drop(ny.index, inplace=True)
si = ny.copy()
si.drop(ny.index, inplace=True)

for idx, x in enumerate(ny.Zip):
    if x in Bronx:
        bx.loc[idx] = ny.loc[idx]
    elif x in Brooklyn:
        bk.loc[idx] = ny.loc[idx]
    elif x in Manhattan:
        man.loc[idx] = ny.loc[idx]
    elif x in Queens:
        que.loc[idx] = ny.loc[idx]
    elif x in StatenIsland:
        si.loc[idx] = ny.loc[idx]

In [25]:
print(f'We have {bx.Zip.nunique()} different zip codes in our dataset for Bronx.')
print(f'We have {bk.Zip.nunique()} different zip codes in our dataset for Brooklyn.')
print(f'We have {man.Zip.nunique()} different zip codes in our dataset for Manhattan.')
print(f'We have {que.Zip.nunique()} different zip codes in our dataset for Queens.')
print(f'We have {si.Zip.nunique()} different zip codes in our dataset for Staten Island.')

We have 13 different zip codes in our dataset for Bronx.
We have 29 different zip codes in our dataset for Brooklyn.
We have 4 different zip codes in our dataset for Manhattan.
We have 56 different zip codes in our dataset for Queens.
We have 12 different zip codes in our dataset for Staten Island.


In [26]:
def melt_data(df):
    ''' 
    Takes a dataframe with datetime data that is in wide format and melts it into long format; 
    Tranforms data into datetime object with time as index.
    User will need to change columns names on first line of code according to their own dataframe.
    '''
    
    melted = pd.melt(df, id_vars=['RegionID', 'Zip', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [70]:
bx_ts = melt_data(bx)
bk_ts = melt_data(bk)
man_ts = melt_data(man)
que_ts = melt_data(que)
si_ts = melt_data(si)
ny_ts = melt_data(ny)

In [71]:
fig = go.Figure()
colorway = ['#f3cec9', '#e7a4b6', '#cd7eaf', '#a262a9', '#6f4d96', '#3d3b72', '#182844']
fig.add_trace(go.Scatter(x=bx_ts.index, y=bx_ts.value, name='Bronx',
                         line_color='green'))
fig.add_trace(go.Scatter(x=bk_ts.index, y=bk_ts.value, name='Brooklyn',
                        line_color='olive'))
fig.add_trace(go.Scatter(x=man_ts.index, y=man_ts.value, name='Manhattan',
                         line_color='red'))
fig.add_trace(go.Scatter(x=que_ts.index, y=que_ts.value, name='Queens',
                         line_color='brown'))
fig.add_trace(go.Scatter(x=si_ts.index, y=si_ts.value, name='Staten Island',
                         line_color='purple'))
fig.add_trace(go.Scatter(x=ny_ts.index, y=ny_ts.value, name='New York City',
                         line_color='black'))
fig.update_layout(title_text='New York City Home Value Index',
                  colorway=colorway, xaxis_rangeslider_visible=True)
fig.show()

In [72]:
def plot_roll(ts, name=''):
    
    '''Takes a time series and plots it along with its rolling mean and its rolling standard deviation.
    Calculates and prints results from Dickey-Fuller test.'''
    
    # Calculate rolling mean and rolling standard deviation:
    rolmean = ts.rolling(window = 6, center = False).mean()
    rolstd = ts.rolling(window = 6, center = False).std()
    
    # Plot original time series and its rolling mean/standard deviation
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ts.index, y=ts.value, name='Original',
                             line_color='blue'))
    fig.add_trace(go.Scatter(x=rolmean.index, y=rolmean.value, name='Rolling Mean',
                             line_color='red'))
    fig.add_trace(go.Scatter(x=rolstd.index, y=rolstd.value, name='Rolling Std',
                             line_color='green'))
    fig.update_layout(title_text=f'{name} Rolling Mean & Standard Deviation',
                      xaxis_rangeslider_visible=True)
    fig.show();
    #Perform Dickey-Fuller test:
    print (f'{name} Results of Dickey-Fuller Test:')
    dftest = adfuller(ts['value'])

    # Extract and display test results in a user friendly manner
    dftest = adfuller(ts['value'])
    # Extract and display test results in a user friendly manner
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return print ('Results of Dickey-Fuller test: ' + str(dfoutput))


In [73]:
plot_roll(bx_ts, 'Bronx')
plot_roll(bk_ts, 'Brooklyn')
plot_roll(man_ts, 'Manhattan')
plot_roll(que_ts, 'Queens')
plot_roll(si_ts, 'Staten Island')
plot_roll(ny_ts, 'New York City')

Bronx Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                  -1.333748
p-value                          0.613595
#Lags Used                       6.000000
Number of Observations Used    258.000000
Critical Value (1%)             -3.455953
Critical Value (5%)             -2.872809
Critical Value (10%)            -2.572775
dtype: float64


Brooklyn Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                   0.509009
p-value                          0.985144
#Lags Used                       8.000000
Number of Observations Used    256.000000
Critical Value (1%)             -3.456155
Critical Value (5%)             -2.872897
Critical Value (10%)            -2.572822
dtype: float64


Manhattan Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                  -0.104723
p-value                          0.948966
#Lags Used                       0.000000
Number of Observations Used    264.000000
Critical Value (1%)             -3.455365
Critical Value (5%)             -2.872551
Critical Value (10%)            -2.572638
dtype: float64


Queens Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                  -1.289382
p-value                          0.634016
#Lags Used                      13.000000
Number of Observations Used    251.000000
Critical Value (1%)             -3.456674
Critical Value (5%)             -2.873125
Critical Value (10%)            -2.572944
dtype: float64


Staten Island Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                  -1.331575
p-value                          0.614606
#Lags Used                      15.000000
Number of Observations Used    249.000000
Critical Value (1%)             -3.456888
Critical Value (5%)             -2.873219
Critical Value (10%)            -2.572994
dtype: float64


New York City Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                   0.573334
p-value                          0.986940
#Lags Used                       0.000000
Number of Observations Used    264.000000
Critical Value (1%)             -3.455365
Critical Value (5%)             -2.872551
Critical Value (10%)            -2.572638
dtype: float64


# Stationary

In [77]:
roll_mean = que_ts.rolling(window=4).mean()
# Subtract the moving average from the original data
que_ts = que_ts - roll_mean
que_ts.head(15)
# Drop the missing values from time series calculated above
que_ts.dropna(inplace=True)
que_ts = que_ts.diff(periods=1)
que_ts.dropna(inplace=True)
plot_roll(que_ts, 'Queens')

Queens Results of Dickey-Fuller Test:
Results of Dickey-Fuller test: Test Statistic                -9.014843e+00
p-value                        5.989174e-15
#Lags Used                     1.600000e+01
Number of Observations Used    2.360000e+02
Critical Value (1%)           -3.458366e+00
Critical Value (5%)           -2.873866e+00
Critical Value (10%)          -2.573339e+00
dtype: float64
